## Width-Scale Bar Chart

References:

- Höhn, M., Wunderlich, M., Ballweg, K., & Landesberger, T. (2020). Width-Scale Bar Charts for Data with Large Value Range. In _EuroVis 2020 - Short Papers_. The Eurographics Association. [🔗](https://diglib.eg.org/handle/10.2312/evs20201056)

In [1]:
import pandas as pd
import altair as alt
from decimal import Decimal

In [2]:
alt.__version__

'4.1.0'

In [3]:
data = dict(
    category=["A", "B", "C", "D", "E", "F", "G", "H"], 
    value=[800, 100, 4, 20, 400, 2000, 10, 60] 
)

In [4]:
def numeric_to_mantissa_and_exponent(value):
    exponent = int(len(str(abs(value)).split(".")[0]) - 1)
    mantissa = float(value * 10 ** -exponent)
    multiplier = int(10 ** exponent)
    
    return mantissa, exponent, multiplier

def end_x(value):
    return 10 * (value + 1)

In [5]:
def create_dataset(data):
    df = pd.DataFrame(data)
    df["mantissa"], df["exponent"], df["multiplier"] = zip(*df["value"].map(numeric_to_mantissa_and_exponent))
    df["end"] = df["exponent"].map(end_x)
    df["end"] = df['end'].cumsum() 
    df["start"] = df["end"].shift(fill_value=0)
    df["middle"] = ((df["end"] - df["start"]) / 2) + df["start"]
    
    return df

In [6]:
df = create_dataset(data)

In [7]:
def wsb_chart(data, w=600, h=300):
    base = alt.Chart(data, width=w, height=h)
    
    bar = base.mark_rect(xOffset=1.0, x2Offset=0.5).encode(
        x=alt.X('start:Q', axis=alt.Axis(titleY=(-0.5 + 22), labels=False, title="Category", grid=False, values=data["middle"].to_list())),
        x2=alt.X2("end:Q"),
        y=alt.Y('mantissa:Q', axis=alt.Axis(titleX=(0.5 + -22), title="Mantissa"), scale=alt.Scale(domain=[0, 10])),
        color=alt.Color("multiplier:O", title="Magnitude Multiplier", legend=alt.Legend(labelExpr="'× ' + datum.value"), scale=alt.Scale(scheme="reds")),
    
    )
    
    # Altair/Vega-Lite:
    # default `fontSize` = 11
    # default `tickSize` = 5
    # default `labelPadding` = 2
    # default `translate` = 0.5
    
    text = base.mark_text(align='center', dx=0.5, baseline='middle', fontSize=11).encode(
        x=alt.X('middle:Q'),
        y=alt.value(h + (11/2) + 5 + 2 + 0.5),
        text=alt.Text("category:N")
    )
    
    return (bar + text).configure_view(strokeWidth=0)

In [8]:
wsb_chart(df)

alt.LayerChart(...)